# RAG Quickstart for Azure AI Search

This quickstart provides a query for RAG scenarios. It demonstrates an approach for a chat experience using grounding data from a search index on Azure AI Search.

We took a few shortcuts to keep the exercise basic and focused on query definitions:

- We use the hotels-sample-index, which can be created in minutes and runs on any search service tier. This index is created by a wizard using built-in sample data.

- We omit vectors so that we can skip chunking and embedding. The index contains plain text.

Once you understand the fundamentals of integrating queries from Azure AI Search to an LLM, you can build on that experience by adding vector fields and vector and hybrid queries. We recommend the [phi-chat Python code example](https://github.com/Azure/azure-search-vector-samples/blob/main/demo-python/code/phi-chat/phi-chat.ipynb) for that step.

This example is fully documented in [Quickstart: Generative search (RAG) with grounding data from Azure AI Search](https://learn.microsoft.com/azure/search/search-get-started-rag). If you need more guidance than the readme provides, please refer to the article.


## Prerequisites

- [Azure OpenAI](https://learn.microsoft.com/azure/ai-services/openai/how-to/create-resource)

  - Deploy a chat model (GPT-3.5-Turbo, GPT-4, or equivalent LLM).

- [Azure AI Search](https://learn.microsoft.com/azure/search/search-create-service-portal)

  - Basic tier or higher is recommended.
  - Choose the same region as Azure OpenAI.
  - Enable semantic ranking.
  - Enable role-based access control.
  - Enable a system identity for Azure AI Search.
  
Make sure you know the name of the deployed model, and have the endpoints for both Azure resources at hand. You will provide this information in the steps that follow.

## Configure access

This quickstart assumes authentication and authorization using Microsoft Entra ID and role assignments. It also assumes that you run this code from your local device.

1. To create, load, and query the sample index on Azure AI Search, you must personally have role assignments for: **Search Index Data Reader**, **Search Index Data Contributor**, **Search Service Contributor**.

1. To send the query and search results to Azure OpenAI, both you and the search system identity must have **Cognitive Services OpenAI User** permissions on Azure OpenAI.

   - Queries in the system prompt are sent from your local system, which is why you need permissions on Azure OpenAI.
   - Results used for grounding data are sent from the search engine, which is why the search service needs permissions on Azure OpenAI.

## Create the sample index

This quickstart assumes the hotels-sample-index, which you can create in minutes using [this quickstart](https://learn.microsoft.com/azure/search/search-get-started-portal).

Once the index exists, modify it in the Azure portal to use this semantic configuration:
  
    ```json
    "semantic": {
    "defaultConfiguration": "semantic-config",
    "configurations": [
      {
        "name": "semantic-config",
        "prioritizedFields": {
          "titleField": { "fieldName": "HotelName" },
          "prioritizedContentFields": [ { "fieldName": "Description" } ],
          "prioritizedKeywordsFields": [
            { "fieldName": "Category" },
            { "fieldName": "Tags" }
          ]}
       }]},
    ```

Now that you have your Azure resources, an index, and model in place, you can run the script to chat with the index.

## Create a virtual environment

Create a virtual environment so that you can install the dependencies in isolation.

1. In Visual Studio Code, press Ctrl-shift-P to open the command palette, search for "Python: Create Environment", and then select `Venv` to create a virtual environment in the current workspace.

1. Select Quickstart-RAG\requirements.txt for the dependencies.

It takes several minutes to create the environment. When the environment is ready, continue to the next step.

## Run the code

In [1]:
# Package install for quickstart
! pip install -r requirements.txt --quiet

In [2]:
# Set endpoints and deployment model (provide the name of the deployment)
AZURE_SEARCH_SERVICE: str = "PUT YOUR SEARCH SERVICE ENDPOINT HERE"
AZURE_OPENAI_ACCOUNT: str = "PUT YOUR AZURE OPENAI ENDPOINT HERE"
AZURE_DEPLOYMENT_MODEL: str = "gpt-35-turbo"

In [3]:
# Set query parameters for grounding the conversation on your search index
search_type="text"
use_semantic_reranker=True
sources_to_include=5

In [5]:
# Set up the query for generating responses
from azure.identity import DefaultAzureCredential
from azure.identity import get_bearer_token_provider
from azure.search.documents import SearchClient
from openai import AzureOpenAI

credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(credential, "https://cognitiveservices.azure.com/.default")
openai_client = AzureOpenAI(
    api_version="2024-06-01",
    azure_endpoint=AZURE_OPENAI_ACCOUNT,
    azure_ad_token_provider=token_provider
)

search_client = SearchClient(
    endpoint=AZURE_SEARCH_SERVICE,
    index_name="hotels-sample-index",
    credential=credential
)

# This prompt provides instructions to the model
GROUNDED_PROMPT="""
You are a friendly assistant that recommends hotels based on activities and amenities.
Answer the query using only the sources provided below in a friendly and concise bulleted manner.
Answer ONLY with the facts listed in the list of sources below.
If there isn't enough information below, say you don't know.
Do not generate answers that don't use the sources below.
Query: {query}
Sources:\n{sources}
"""

# Query is the question being asked. It's sent to the search engine and the LLM.
query="Can you recommend a few hotels near the ocean with beach access and good views"

# Set up the search results and the chat thread.
# Retrieve the selected fields from the search index related to the question.
search_results = search_client.search(
    search_text=query,
    top=5,
    select="Description,HotelName,Tags"
)
sources_formatted = "\n".join([f'{document["HotelName"]}:{document["Description"]}:{document["Tags"]}' for document in search_results])

response = openai_client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": GROUNDED_PROMPT.format(query=query, sources=sources_formatted)
        }
    ],
    model=AZURE_DEPLOYMENT_MODEL
)

print(response.choices[0].message.content)

Sure, here are a few hotels that match your criteria: 

- Ocean Air Motel, which is an oceanfront hotel with beach access, private balconies, indoor and outdoor pools, shops, and art entertainment nearby.
- Marquis Plaza & Suites, which offers ocean views, free Wi-Fi, a full kitchen, and free breakfast buffet, along with beach access and a pool.
- Oceanside Resort is a new luxury hotel with bay views from every room, a rooftop pool, waterfront dining, and laundry service. 

Unfortunately, the Double Sanctuary Resort and Triple Landscape Hotel don't have beach access or mention views.


If you get an authorization error instead of results, wait a few minutes and try again. It can take several minutes for role assignments to become operational.